In [ ]:
import pandas as pd
import streamlit as st
import plotly.graph_objects as go
from .processing import (
    preparar_dataframes,
    aplicar_transformaciones,
    procesar_picking,
    procesar_chequeo,
    unir_datos,
    create_grouped_report,
    PK_IMAGEN,
    ORDER_EMPRESAS
)

def format_dataframe(df):
    """Da formato al DataFrame para la visualización."""
    column_mapping = {
        'Empresa': 'EMPRESA',
        'empresa': 'EMPRESA',
        'Rendimiento': 'RENDIMIENTO',
        'rendimiento': 'RENDIMIENTO',
        'Cjs c/ Error': 'CJS C/ ERROR',
        '% Error': '% ERROR',
        'Descripcion': 'Descripcion'
    }
    
    for old_name, new_name in column_mapping.items():
        if old_name in df.columns:
            df = df.rename(columns={old_name: new_name})
    
    desired_columns = [
        'EMPRESA',
        'USUARIO',
        'Descripcion',
        'CAJAS',
        'RENDIMIENTO',
        '% ERROR',
        'CJS C/ ERROR'
    ]
    
    columns_to_use = [col for col in desired_columns if col in df.columns]
    df = df[columns_to_use]
    return df

def sort_dataframe(df):
    """Ordena el DataFrame según los requisitos específicos."""
    df = df.copy()
    
    df['empresa_order'] = pd.Categorical(
        df['EMPRESA'],
        categories=ORDER_EMPRESAS,
        ordered=True
    )
    
    df['row_type'] = 'normal'
    df.loc[df['USUARIO'].str.startswith('Total '), 'row_type'] = 'subtotal'
    df.loc[df['EMPRESA'] == 'TOTAL GENERAL', 'row_type'] = 'total'
    
    df_sorted = df.sort_values(
        by=['empresa_order', 'row_type', 'CAJAS'],
        ascending=[True, True, False],
        key=lambda x: pd.Categorical(x, categories=['normal', 'subtotal', 'total'], ordered=True) if x.name == 'row_type' else x
    )
    
    df_sorted = df_sorted.drop(['empresa_order', 'row_type'], axis=1)
    return df_sorted

def highlight_cells(df, rendimiento_min, rendimiento_mediana, rendimiento_max):
    """Aplica estilos condicionales al dataframe."""
    def apply_styles(row):
        is_total_row = "Total" in str(row['USUARIO']) or row['EMPRESA'] == "TOTAL GENERAL"
        is_pk_imagen = str(row['USUARIO']) in PK_IMAGEN
        
        if is_total_row:
            return ['background-color: rgb(166,166,166); color: black; font-weight: bold'] * len(df.columns)
        
        row_styles = ['color: black'] * len(df.columns)
        
        if is_pk_imagen:
            rendimiento_idx = df.columns.get_loc('RENDIMIENTO')
            row_styles[rendimiento_idx] = 'background-color: #CCCCCC; color: #777777; font-style: italic'
        elif pd.notna(row['RENDIMIENTO']):
            rendimiento_idx = df.columns.get_loc('RENDIMIENTO')
            if row['RENDIMIENTO'] <= rendimiento_min:
                row_styles[rendimiento_idx] = 'background-color: #FF9999; color: black'
            elif row['RENDIMIENTO'] <= rendimiento_mediana:
                row_styles[rendimiento_idx] = 'background-color: #FFFF99; color: black'
            else:
                row_styles[rendimiento_idx] = 'background-color: #90EE90; color: black'
        
        if pd.notna(row['% ERROR']):
            error_idx = df.columns.get_loc('% ERROR')
            if row['% ERROR'] > 0.05:
                row_styles[error_idx] = 'background-color: #FF9999; color: black'
            else:
                row_styles[error_idx] = 'background-color: #90EE90; color: black'
        
        return row_styles
    
    return df.style.apply(apply_styles, axis=1).format({
        '% ERROR': "{:.2f} %",
        'RENDIMIENTO': "{:.2f}",
        'CJS C/ ERROR': "{:,.0f}",
        'CAJAS': "{:,.0f}"
    }, na_rep="N/A")

def create_kpi_metrics(df_final, meta_rendimiento=310, meta_error=0.05):
    """Crea los indicadores KPI."""
    try:
        rendimiento_general = df_final[~df_final['USUARIO'].isin(PK_IMAGEN)]['RENDIMIENTO'].mean()
        pct_rendimiento = (rendimiento_general / meta_rendimiento) * 100
        error_general = df_final['% ERROR'].mean()
        pct_error = (1 - (error_general / meta_error)) * 100
        
        return {
            'rendimiento_actual': rendimiento_general,
            'pct_rendimiento': pct_rendimiento,
            'error_actual': error_general,
            'pct_error': pct_error
        }
    except Exception as e:
        st.error(f"Error al calcular KPIs: {str(e)}")
        return None

def create_nivel_carga_summary(df_picking):
    """Crea el resumen de nivel de carga."""
    try:
        pivot_nivel_carga = pd.pivot_table(
            df_picking,
            values='CAJAS',
            index='EMPRESA',
            columns='Fecha Entrega',
            aggfunc='sum',
            fill_value=0
        )
        pivot_nivel_carga.loc['Total general'] = pivot_nivel_carga.sum()
        return pivot_nivel_carga
    except Exception as e:
        st.error(f"Error en nivel de carga: {str(e)}")
        return pd.DataFrame()

def create_descuento_summary(df_chequeo):
    """Crea el resumen de descuentos."""
    try:
        if 'CJS C/ ERROR' not in df_chequeo.columns:
            return pd.DataFrame()
            
        # Filtrar registros con errores
        df_malo = df_chequeo[df_chequeo['CJS C/ ERROR'] > 0].copy()
        
        # Crear tabla dinámica
        pivot_descuento = pd.pivot_table(
            df_malo,
            values='CJS C/ ERROR',
            index=['USUARIO'],
            aggfunc='sum',
            fill_value=0
        )
        
        # Ordenar por cantidad de errores descendente
        return pivot_descuento.sort_values('CJS C/ ERROR', ascending=False)
        
    except Exception as e:
        st.error(f"Error en descuentos: {str(e)}")
        return pd.DataFrame()

def main():
    """Función principal de visualización."""
    try:
        st.title('Rendimiento de Producción')
        
        # Cargar y procesar datos
        df_picking, df_chequeo = preparar_dataframes()
        
        if df_picking is not None and df_chequeo is not None:
            # Procesar datos
            df_picking, df_chequeo = aplicar_transformaciones(df_picking, df_chequeo)
            df_valid = procesar_picking(df_picking)
            pallet_grouped = procesar_chequeo(df_chequeo)
            df_final = unir_datos(df_valid, pallet_grouped)
            
            # Obtener fecha y KPIs
            fecha_reporte = df_picking['Fecha Entrega'].iloc[0]
            kpis = create_kpi_metrics(df_final)
            
            # 1. Métricas principales
            col1, col2, col3, col4 = st.columns(4)
            col1.metric("Fecha de Reporte", fecha_reporte)
            
            if kpis:
                col2.metric("Rendimiento General",
                          f"{kpis['rendimiento_actual']:.2f}",
                          f"{kpis['pct_rendimiento']:.1f}% de meta (310)")
                
                col3.metric("Error General",
                          f"{kpis['error_actual']:.2f}%",
                          f"{kpis['pct_error']:.1f}% de meta (0.05%)",
                          delta_color="inverse")
            
            col4.metric("Total Cajas", f"{df_final['CAJAS'].sum():,.0f}")
            
            st.markdown("---")
            
            # 2. Resúmenes
            col1, col2 = st.columns(2)
            
            with col1:
                st.markdown("### Nivel de Carga")
                nivel_carga = create_nivel_carga_summary(df_picking)
                if not nivel_carga.empty:
                    st.dataframe(nivel_carga.style.format("{:,.0f}"))
            
            with col2:
                st.markdown("### Detalle de Descuentos")
                descuento_summary = create_descuento_summary(df_final)
                if not descuento_summary.empty:
                    st.dataframe(descuento_summary.style.format("{:,.0f}"))
            
            # 3. Tabla principal
            st.markdown("---")
            st.markdown("### Reporte Detallado")
            
            df_final_report, min_rendimiento, mediana_rendimiento, max_rendimiento = create_grouped_report(df_final)
            df_final_report = format_dataframe(df_final_report)
            df_final_report = sort_dataframe(df_final_report)
            
            styled_df = highlight_cells(df_final_report,
                                      min_rendimiento,
                                      mediana_rendimiento,
                                      max_rendimiento)
            
            st.dataframe(styled_df, height=800)
        
    except Exception as e:
        st.error(f"Error en la visualización: {str(e)}")
        st.exception(e)

if __name__ == "__main__":
    main()